<a href="https://colab.research.google.com/github/xuwangfmc/dlbook/blob/main/modelcompression/ArchitectureDesign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 架构设计

模型设计（Architecture Designing）的思想就是设计轻量化的网络结构以减少模型参数和加快模型运算的速度。常见的轻量化模型有MobileNet和ShuffleNet等系列，其中Depthwise Convolution、Pointwise Convolution、Group Convolution是常用的基本操作。 
  
- Depthwise Convolution 就是对每张feature map单独用一个filter处理进行处理。后面t通常会会接一个Pointwise Convolution将处理后的信息融合在一起。这两种卷积方式合起来也叫Depth Separable Convolution。

- Pointwise Convolution就是对每张feature map采用$1\times 1$的卷积核进行处理，再融合所有处理后的信息。

- Group Convolution就是把feature map分组，让它们自己过Convolution Layer后再重新Concat起来。算是Standard Convolution和Depthwise Convolution的折中版。所以说，Group Convolution的Group=Input Features数就会使Depthwise Convolution(因为每个Channel都各自独立)，Group=1就会是Standard Convolution(因为等于没有Group)。 

- Standard Convolution 是一般采用的卷积，其Weight连接方式会跟Fully Connected一样，只差在原本在FC是用数字相乘后相加，Convolution Layer是图片卷积后相加。

Standard Convolution与Group Convolution的区别如下图所示：
![Hqhg0Q9.png](https://s2.loli.net/2022/01/22/XcfvEjwHoVJeDbY.png)

该教程主要介绍以上四种卷积在Pytorch中的定义方式，以及如何使用Depthwise Convolution和Pointwise Convolution来构建自己的模型。

## 四种卷积在Pytorch中的定义方式


```python

# Standard Convolution, weight大小 = in_chs * out_chs * kernel_size^2
nn.Conv2d(in_chs, out_chs, kernel_size, stride, padding)
 
# Group Convolution, Group数目可以自行控制，表示要分成几群。其中in_chs和out_chs必需要可以被groups整除。
nn.Conv2d(in_chs, out_chs, kernel_size, stride, padding, groups=groups)
 
# Depthwise Convolution, 输入chs=输出chs=Groups数目, weight大小 = in_chs * kernel_size^2
nn.Conv2d(in_chs, out_chs=in_chs, kernel_size, stride, padding, groups=in_chs)
 
# Pointwise Convolution, 也就是1 by 1 convolution, weight大小 = in_chs * out_chs
nn.Conv2d(in_chs, out_chs, 1)

```




## 实战案例

In [ ]:
import torch.nn as nn


class StudentNet(nn.Module):
    def __init__(self, base=16, width_mult=1):
        '''
          Args:
            base: 这个model一开始的channel数量
            width_mult: 为了之后能进行剪枝，在一些层上会乘以width_mult代表剪枝后的通道数量。width_mult为1时表示不剪枝。
        '''
        super(StudentNet, self).__init__()
        multiplier = [1, 2, 4, 8, 16, 16, 16, 16]
        bandwidth = [ base * m for m in multiplier] # 每层输出的channel数量
        for i in range(3, 7): # 对3/4/5/6层进行剪枝
            bandwidth[i] = int(bandwidth[i] * width_mult)

        self.cnn = nn.Sequential(
            # 我们通常不会拆解第一个卷积
            nn.Sequential(
                nn.Conv2d(3, bandwidth[0], 3, 1, 1),
                nn.BatchNorm2d(bandwidth[0]),
                nn.ReLU6(),
                nn.MaxPool2d(2, 2, 0)
            ),

            # 接下来的每个Sequential都一样，所以只详细介绍接下来第一个Sequential
            nn.Sequential(
                # Depthwise Convolution
                nn.Conv2d(bandwidth[0], bandwidth[0], 3, 1, 1, groups=bandwidth[0]),
                # Batch Normalization
                nn.BatchNorm2d(bandwidth[0]),
                # ReLU6是限制神经元最小为0最大为6，MobileNet系列都是使用ReLU6。原因是如果数字太大，会不好压到float16 / or further qunatization。
                nn.ReLU6(),
                # Pointwise Convolution，之后不需要再做ReLU，经验上Pointwise + ReLU效果都会变化。
                nn.Conv2d(bandwidth[0], bandwidth[1], 1),
                # 每过完一个Block就下采样
                nn.MaxPool2d(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2d(bandwidth[1], bandwidth[1], 3, 1, 1, groups=bandwidth[1]),
                nn.BatchNorm2d(bandwidth[1]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[1], bandwidth[2], 1),
                nn.MaxPool2d(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2d(bandwidth[2], bandwidth[2], 3, 1, 1, groups=bandwidth[2]),
                nn.BatchNorm2d(bandwidth[2]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[2], bandwidth[3], 1),
                nn.MaxPool2d(2, 2, 0),
            ),

            # 目前图片已经进行了多次下采样，所以就不再做MaxPool
            nn.Sequential(
                nn.Conv2d(bandwidth[3], bandwidth[3], 3, 1, 1, groups=bandwidth[3]),
                nn.BatchNorm2d(bandwidth[3]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[3], bandwidth[4], 1),
            ),

            nn.Sequential(
                nn.Conv2d(bandwidth[4], bandwidth[4], 3, 1, 1, groups=bandwidth[4]),
                nn.BatchNorm2d(bandwidth[4]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[4], bandwidth[5], 1),
            ),

            nn.Sequential(
                nn.Conv2d(bandwidth[5], bandwidth[5], 3, 1, 1, groups=bandwidth[5]),
                nn.BatchNorm2d(bandwidth[5]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[5], bandwidth[6], 1),
            ),

            nn.Sequential(
                nn.Conv2d(bandwidth[6], bandwidth[6], 3, 1, 1, groups=bandwidth[6]),
                nn.BatchNorm2d(bandwidth[6]),
                nn.ReLU6(),
                nn.Conv2d(bandwidth[6], bandwidth[7], 1),
            ),

            # 如果输入图片大小不同，Global Average Pooling会把它们压成相同形状，这样接下来FC就不会出问题
            nn.AdaptiveAvgPool2d((1, 1))
        )

        self.fc = nn.Sequential(
            nn.Linear(bandwidth[7], 11)  # 直接将CNN的输出映射到11维作为最终输出
        )


    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
